# Installation of JanusQ
JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended as all functions have been tested on the docker.
Linux (xxx) platform and Python (xxx) is prefered. Otherwise, some functions may not be available.

**From docker.**
Pull docker using docker 
        
        pull jxhhhh/janusq:0.1

The code is in "/home/JanusQ-main". The examples that can be directly run is in "/home/JanusQ-main/examples"

**From wheel.**
Download janusq.whl from "xxxx".

        pip install janusq.whl

**From source code.**
Run following commends.

        git clone git@github.com:JanusQ/JanusQ.git
        cd JanusQ
        pip install -r requirements.txt

# Structure of JanusQ

TODO: 看看artifact写一下

# Test JanusQ

In [9]:
import sys
sys.path.append('..')

#### Test Janus-CT

In [10]:
# test janus-ct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel
# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
vec_model.train(circuits, multi_process=False, remove_redundancy = False)

2024-04-14 13:55:00,424 - INFO: start random walk for 300 circuits
100%|██████████| 300/300 [00:02<00:00, 131.18it/s]
2024-04-14 13:55:02,713 - INFO: count path
2024-04-14 13:55:02,747 - INFO: device size after random walk = 15
2024-04-14 13:55:02,747 - INFO: 0's path table size = 11
2024-04-14 13:55:02,748 - INFO: 1's path table size = 16
2024-04-14 13:55:02,748 - INFO: 2's path table size = 19
2024-04-14 13:55:02,749 - INFO: 3's path table size = 19
2024-04-14 13:55:02,749 - INFO: 4's path table size = 19
2024-04-14 13:55:02,749 - INFO: 5's path table size = 19
2024-04-14 13:55:02,749 - INFO: 6's path table size = 16
2024-04-14 13:55:02,750 - INFO: 7's path table size = 11
2024-04-14 13:55:02,751 - INFO: (0, 1)'s path table size = 13
2024-04-14 13:55:02,751 - INFO: (1, 2)'s path table size = 18
2024-04-14 13:55:02,751 - INFO: (3, 4)'s path table size = 21
2024-04-14 13:55:02,752 - INFO: (2, 3)'s path table size = 21
2024-04-14 13:55:02,752 - INFO: (6, 7)'s path table size = 13
2024-0

#### Test Janus-FEM

In [7]:
# test readout calibration

from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algs

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algs(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


2024-04-14 13:10:32,850 - DEBUG: found extension EntryPoint(name='default', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DefaultInitPassManager', group='qiskit.transpiler.init')
2024-04-14 13:10:32,852 - DEBUG: found extension EntryPoint(name='default', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DefaultLayoutPassManager', group='qiskit.transpiler.layout')
2024-04-14 13:10:32,852 - DEBUG: found extension EntryPoint(name='dense', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DenseLayoutPassManager', group='qiskit.transpiler.layout')
2024-04-14 13:10:32,853 - DEBUG: found extension EntryPoint(name='noise_adaptive', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:NoiseAdaptiveLayoutPassManager', group='qiskit.transpiler.layout')
2024-04-14 13:10:32,854 - DEBUG: found extension EntryPoint(name='sabre', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:SabreLayoutPassManager', group='qiskit.transpiler.layout')
2024-04-1

[{'100': 19, '110': 2, '011': 65, '001': 643, '010': 800, '000': 8471}, {'101': 15, '010': 48, '000': 466, '111': 1, '011': 815, '001': 8655}, {'101': 1, '110': 4, '011': 46, '100': 20, '001': 611, '010': 806, '000': 8512}, {'100': 1, '110': 16, '001': 46, '111': 2, '011': 648, '000': 602, '010': 8685}, {'101': 1, '110': 3, '000': 35, '010': 495, '111': 14, '011': 8847, '001': 605}, {'100': 2, '101': 1, '110': 26, '001': 48, '111': 3, '011': 643, '000': 552, '010': 8725}, {'110': 4, '101': 1, '100': 20, '001': 630, '011': 50, '000': 8503, '010': 792}, {'101': 20, '011': 800, '010': 45, '000': 473, '001': 8662}, {'100': 10, '011': 60, '101': 3, '001': 634, '000': 8484, '010': 809}, {'001': 6, '111': 71, '010': 7, '000': 58, '101': 602, '110': 808, '100': 8448}, {'000': 5, '001': 41, '110': 55, '100': 507, '011': 7, '111': 841, '101': 8544}, {'001': 3, '011': 1, '111': 55, '000': 60, '010': 6, '110': 805, '101': 644, '100': 8426}, {'100': 599, '000': 2, '010': 49, '101': 47, '011': 1, '1

#### Test Janus-SAT

In [11]:
# test hyqsat calibration
# input cnf flie
file_path = "cnf_examples/uf50-01.cnf"
from janusq.hyqsat import readCNF

print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -